In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [2]:
import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight

from inception_model_generator import MRNet_inc_model


Using TensorFlow backend.


In [0]:
label_type = 'abnormal'
exam_view = 'axial'
model, cbs = MRNet_inc_model(combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
weight_for_0 = (1 / neg)*(total) 
weight_for_1 = (1 / pos)*(total)
class_weight_train = {0: weight_for_0, 1: weight_for_1}

pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
weight_for_0 = (1 / neg)*(total) 
weight_for_1 = (1 / pos)*(total)
class_weight_test = {0: weight_for_0, 1: weight_for_1}
batch_size = 1

train_data_gen = MRNet_data_generator(data_path, p, l, batch_size=1,model="incept",  data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0, class_weight=class_weight_train)
test_data_gen = MRNet_data_generator(data_path, p, l, batch_size=1,model="incept",  data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0, class_weight=class_weight_test)
# model.load_weights('/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/abnormal/axial/weights.05.hdf5')
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=20, initial_epoch = 0,verbose=True, callbacks=[cbs],shuffle=False, use_multiprocessing=True, workers=6)

In [0]:
b_size = 1

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)


In [0]:

valid_gen = MRNet_data_generator(data_path, p, l, batch_size=b_size, scale_to=(299,299),train=False, shuffle=False,model="inc",aug_size=0)
print(valid_gen.__len__());

In [0]:
model, cp_callback = MRNet_inc_model(["abnormal", "axial"])
os.chdir('/content/gdrive/My Drive/Colab Notebooks/MRNet')

model.load_weights('training_inception/abnormal/axial/cp.ckpt');


In [0]:
d = model.evaluate(x=valid_gen,verbose=1,use_multiprocessing=True,workers=6);
print(d)


In [0]:
model.metrics_names

In [0]:

d = model.predict(x=valid_gen,verbose=1,use_multiprocessing=True,workers=6)
print(d)